# Knn图形识别

In [1]:
import numpy as np
import pandas as pd
import operator
from os import listdir

In [2]:
#定义一个打开文件夹中test文件的函数，并且转化成一维向量
def onearray(filename):
    returnVect = np.zeros((1,1024))  #待转化成这种格式，原格式为32*32
    fr = open(filename)
    for i in range(32):  #读取第i行
        linestr = fr.readline()
        for j in range(32):
            returnVect[0,i*32 + j] = int(linestr[j])  #要转化成整型，将第i行第j个数据填到一维向量中的i*32 + j位置中
    return returnVect


In [3]:
#定义KNN算法
def knn(x_train,x_test,y_train,k):
    x_train_n = x_train.shape[0]  #训练样本个数
    diff = np.tile(x_test,(x_train_n,1)) - x_train
    pf = diff**2
    pfh = pf.sum(axis = 1)
    distances = pfh**0.5 #计算x_test与各个train的欧式距离
    sortedDistIndicies = distances.argsort()    # 返回排序从小到达的索引位置
    classCount = {}   # 字典存储k近邻不同Y出现的次数，用来进行投票
    for i in range(k):  #准备找k个临近点
        votelabel = y_train[sortedDistIndicies[i]]
        classCount[votelabel] = classCount.get(votelabel, 0) + 1    # 对应label加1，classCount中若无此key，则默认为0
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True) 
    # operator.itemgetter 获取对象的哪个维度的数据
    return sortedClassCount[0][0] #取最前面的那个为投票最多的值Y
    

In [4]:
#进行样本的训练以及预测
def Classtest():
    y_train = [] 
    trainingFileList = listdir('trainingDigits')  #读取训练集样本的文件的名字
    m = len(trainingFileList)  #读取文件夹中数据txt的个数
    x_train = np.zeros((m,1024))  #构建一整块的数据集
    for i in range(m):
        filenamestr = trainingFileList[i] #得到每个文件的名字
        filestr = filenamestr.split('.')[0]
        y_traini = int(filestr.split('_')[0])
        y_train.append(y_traini)  #文件的第一个字母为类别
        x_train[i,:] = onearray('trainingDigits/%s' % filenamestr) #读取每一个文件，并且化成一维向量
    #这时候trainmat为是一整块数据集了，x_train与y_train都准备好了
    #现在要准备x_train了
    testFileList = listdir('testDigits')
    m_test = len(testFileList)
    y_test = []
    y_pre = []
    errorCount = 0.0
    for j in range(m_test):
        filenamestr = testFileList[j]
        filestr = filenamestr.split('.')[0]
        y_testi = int(filestr.split('_')[0])
        y_test.append(y_testi)
        x_testj = onearray('testDigits/%s' %filenamestr)
        #对每个j样本，都要作KNN算法了
        y_pre_j = knn(x_train,x_testj,y_train,3)
        y_pre.append(y_pre_j)
        if (y_pre_j != y_testi): errorCount += 1.0
    score = np.mean(y_test == y_pre)
    #print(score)
    #print(y_pre)
    #print(y_test)
    print(errorCount)
    
    

In [5]:
Classtest()

10.0
